In [1]:
import yaml
import torch
import time


from models import GAT, GCN, SWEGNN
from models.swe_gnn_ned import SWEGNN_NED
from data import TemporalGraphDataset

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
with open('config.yaml') as f:
    config = yaml.safe_load(f)

dataset, info = TemporalGraphDataset(node_features=config['node_features'],
                    edge_features=config['edge_features'],
                    **config['dataset_parameters']).load()

In [4]:
print(dataset[0])
print(type(dataset[0].x), dataset[0].x.shape)
print(type(dataset[0].edge_index), dataset[0].edge_index.shape)
print(type(dataset[0].edge_attr), dataset[0].edge_attr.shape)
print(type(dataset[0].y), dataset[0].y.shape)
print(info)

Data(x=[1268, 6], edge_index=[2, 2612], edge_attr=[2612, 8], y=[1268, 1], pos=[2, 1268])
<class 'torch.Tensor'> torch.Size([1268, 6])
<class 'torch.Tensor'> torch.Size([2, 2612])
<class 'torch.Tensor'> torch.Size([2612, 8])
<class 'torch.Tensor'> torch.Size([1268, 1])
{'num_static_node_features': 3, 'num_dynamic_node_features': 1, 'num_static_edge_features': 5, 'num_dynamic_edge_features': 1, 'previous_timesteps': 2}


In [5]:
num_train = int(len(dataset) * 0.8) # 80% train, 20% test

train_dataset = dataset[:num_train]
# train_loader = DataLoader(train_dataset) # batch_size=32, shuffle=True

test_dataset = dataset[num_train:]
# test_loader = DataLoader(test_dataset) # batch_size=32, shuffle=True

In [6]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
base_model_params = {
    'static_node_features': info['num_static_node_features'],
    'dynamic_node_features': info['num_dynamic_node_features'],
    'static_edge_features': info['num_static_edge_features'],
    'dynamic_edge_features': info['num_dynamic_edge_features'],
    'previous_timesteps': info['previous_timesteps'],
    'device': device,
}
lr_info = config['training_parameters']
model_info = config['model_parameters']

In [7]:
def train(model, loss_func, optimizer):
    start_time = time.time()
    num_epochs = 10
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for graph in train_dataset:
            graph = graph.to(device)
            labels = graph.y

            optimizer.zero_grad()

            outputs = model(graph)

            loss = loss_func(outputs, labels)
            running_loss += loss.item()

            loss.backward()
            optimizer.step()

        epoch_loss = running_loss / num_train
        print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {epoch_loss:.4f}')
    end_time = time.time()
    print(f'Total training time: {end_time - start_time} seconds')


def test(model, loss_func):
    start_time = time.time()
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for graph in test_dataset:
            graph = graph.to(device)
            labels = graph.y

            outputs = model(graph)

            loss = loss_func(outputs, labels)
            running_loss += loss.item()
    end_time = time.time()

    # Print validation statistics
    print(f'Validation Loss: {running_loss:.4f}')
    print(f'Inference time: {end_time - start_time} seconds')

In [8]:
gcn_params = model_info['GCN']
model = GCN(**gcn_params, **base_model_params)
optimizer = torch.optim.Adam(model.parameters(), lr=lr_info['learning_rate'], weight_decay=lr_info['weight_decay'])
loss_func = torch.nn.L1Loss()

train(model, loss_func, optimizer)

Epoch [1/10], Training Loss: 4.9178
Epoch [2/10], Training Loss: 0.0215
Epoch [3/10], Training Loss: 0.0206
Epoch [4/10], Training Loss: 12.7546
Epoch [5/10], Training Loss: 0.2981
Epoch [6/10], Training Loss: 0.0434
Epoch [7/10], Training Loss: 0.0298
Epoch [8/10], Training Loss: 0.0277
Epoch [9/10], Training Loss: 0.0260
Epoch [10/10], Training Loss: 0.0237
Total training time: 16.24303364753723 seconds


In [9]:
test(model, loss_func)

Validation Loss: 0.3263
Inference time: 0.16010379791259766 seconds


In [10]:
gat_params = model_info['GAT']
model = GAT(**gat_params, **base_model_params)
optimizer = torch.optim.Adam(model.parameters(), lr=lr_info['learning_rate'], weight_decay=lr_info['weight_decay'])
loss_func = torch.nn.L1Loss()

train(model, loss_func, optimizer)

Epoch [1/10], Training Loss: 17.2771
Epoch [2/10], Training Loss: 0.0395
Epoch [3/10], Training Loss: 0.0230
Epoch [4/10], Training Loss: 0.0215
Epoch [5/10], Training Loss: 0.0209
Epoch [6/10], Training Loss: 0.0412
Epoch [7/10], Training Loss: 0.0208
Epoch [8/10], Training Loss: 0.0208
Epoch [9/10], Training Loss: 0.0210
Epoch [10/10], Training Loss: 0.0213
Total training time: 23.81217122077942 seconds


In [11]:
test(model, loss_func)

Validation Loss: 0.2307
Inference time: 0.26607489585876465 seconds


In [13]:
swe_gnn_params = model_info['SWEGNN']
model = SWEGNN(**swe_gnn_params, **base_model_params)
optimizer = torch.optim.Adam(model.parameters(), lr=lr_info['learning_rate'], weight_decay=lr_info['weight_decay'])
loss_func = torch.nn.L1Loss()

train(model, loss_func, optimizer)

Epoch [1/10], Training Loss: 0.0811
Epoch [2/10], Training Loss: 0.0206
Epoch [3/10], Training Loss: 0.0206
Epoch [4/10], Training Loss: 0.0206
Epoch [5/10], Training Loss: 0.0206
Epoch [6/10], Training Loss: 0.0206
Epoch [7/10], Training Loss: 0.0210
Epoch [8/10], Training Loss: 0.1796
Epoch [9/10], Training Loss: 0.0209
Epoch [10/10], Training Loss: 0.0222
Total training time: 162.76097869873047 seconds


In [14]:
test(model, loss_func)

Validation Loss: 0.1828
Inference time: 1.228783369064331 seconds


In [15]:
# No encoder decoder
swe_gnn_params = model_info['SWEGNN']
swe_gnn_params['encoder_layers'] = 0
swe_gnn_params['encoder_activation'] = None
swe_gnn_params['decoder_layers'] = 0
swe_gnn_params['decoder_activation'] = None

model = SWEGNN(**swe_gnn_params, **base_model_params)
optimizer = torch.optim.Adam(model.parameters(), lr=lr_info['learning_rate'], weight_decay=lr_info['weight_decay'])
loss_func = torch.nn.L1Loss()

train(model, loss_func, optimizer)

Epoch [1/10], Training Loss: 7.0607
Epoch [2/10], Training Loss: 0.0817
Epoch [3/10], Training Loss: 0.0781
Epoch [4/10], Training Loss: 0.0842
Epoch [5/10], Training Loss: 0.0673
Epoch [6/10], Training Loss: 0.0546
Epoch [7/10], Training Loss: 0.0517
Epoch [8/10], Training Loss: 0.0471
Epoch [9/10], Training Loss: 0.0428
Epoch [10/10], Training Loss: 0.0477
Total training time: 127.4778802394867 seconds


In [16]:
test(model, loss_func)

Validation Loss: 3.2959
Inference time: 1.157179594039917 seconds


## Self-Supervised Learning Methods